## Initialise SparkSession and Load Data

In [1]:
from pyspark.sql import SparkSession
# initialise sparkContext
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
import pyspark
import pyarrow
import pandas as pd

In [3]:
dataset= spark.read.load("H1full.csv", format="csv", header="true", inferSchema=True)
dataset
cols = dataset.columns

In [4]:
type(dataset)

pyspark.sql.dataframe.DataFrame

## Print Schema

In [5]:
dataset.printSchema()

root
 |-- IsCanceled: integer (nullable = true)
 |-- LeadTime: integer (nullable = true)
 |-- ArrivalDateYear: integer (nullable = true)
 |-- ArrivalDateMonth: string (nullable = true)
 |-- ArrivalDateWeekNumber: integer (nullable = true)
 |-- ArrivalDateDayOfMonth: integer (nullable = true)
 |-- StaysInWeekendNights: integer (nullable = true)
 |-- StaysInWeekNights: integer (nullable = true)
 |-- Adults: integer (nullable = true)
 |-- Children: integer (nullable = true)
 |-- Babies: integer (nullable = true)
 |-- Meal: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- MarketSegment: string (nullable = true)
 |-- DistributionChannel: string (nullable = true)
 |-- IsRepeatedGuest: double (nullable = true)
 |-- PreviousCancellations: integer (nullable = true)
 |-- PreviousBookingsNotCanceled: integer (nullable = true)
 |-- ReservedRoomType: string (nullable = true)
 |-- AssignedRoomType: string (nullable = true)
 |-- BookingChanges: integer (nullable = true)
 |-- Depos

## Pipeline and Data Transformation

In [6]:
# https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
# https://docs.databricks.com/applications/machine-learning/mllib/binary-classification-mllib-pipelines.html

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ["Country", "MarketSegment", "ArrivalDateMonth", "DepositType", "CustomerType"]

stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

## Convert label into label indices using the StringIndexer

In [7]:
label_stringIdx = StringIndexer(inputCol="IsCanceled", outputCol="label")
stages += [label_stringIdx]

## Transform all features into a vector using VectorAssembler

In [8]:
numericCols = ["LeadTime", "ArrivalDateYear", "ArrivalDateWeekNumber", "ArrivalDateDayOfMonth", "RequiredCarParkingSpaces"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

## GBTClassifier

In [9]:
from pyspark.ml.classification import GBTClassifier
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [10]:
gbtClassifier = GBTClassifier()
trainedModel = gbtClassifier.fit(preppedDataDF)

In [11]:
display(gbtClassifier, preppedDataDF)

GBTClassifier_a5b5a190e841

DataFrame[IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: int, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp, CountryIndex: double, CountryclassVec: vector, MarketSegmentIndex: double, MarketSegmentclassVec: vector, ArrivalDateMonthIndex: double, ArrivalDateMonthclassVec: vector, DepositTypeIndex: double, DepositTypeclassVec: vector, CustomerTypeIndex: double, CustomerTypeclassVec: vector, lab

In [12]:
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)

DataFrame[label: double, features: vector, IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: int, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp]

## Train-Test Split

In [13]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

28056
12004


In [14]:
lrModel = gbtClassifier.fit(trainingData)

In [15]:
predictions = lrModel.transform(testData)
predictions

DataFrame[label: double, features: vector, IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: int, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp, rawPrediction: vector, probability: vector, prediction: double]

In [16]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

## Model Evaluation

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.9163688694820049

In [18]:
evaluator.getMetricName()

'areaUnderROC'

In [19]:
print(lrModel.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: all)
featuresCol: features column name (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (undefined)
labelCol: label column name (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
ma

In [20]:
predictions = lrModel.transform(testData)

In [21]:
evaluator.evaluate(predictions)

0.9163688694820049

In [22]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

In [23]:
type(selected)

pyspark.sql.dataframe.DataFrame

In [24]:
selected.toPandas().to_csv('predictionsh1_gbt.csv')

## H2: Predictions on Test Data

In [25]:
datasettwo= spark.read.load("H2full.csv", format="csv", header="true", inferSchema=True)
datasettwo

pipelineModel = partialPipeline.fit(datasettwo)
preppedDataDF = pipelineModel.transform(datasettwo)

In [26]:
preppedDataDF

DataFrame[IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: string, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp, CountryIndex: double, CountryclassVec: vector, MarketSegmentIndex: double, MarketSegmentclassVec: vector, ArrivalDateMonthIndex: double, ArrivalDateMonthclassVec: vector, DepositTypeIndex: double, DepositTypeclassVec: vector, CustomerTypeIndex: double, CustomerTypeclassVec: vector, 

In [27]:
selectedcols = ["label", "features"] + cols
datasettest = preppedDataDF.select(selectedcols)
display(datasettest)

DataFrame[label: double, features: vector, IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: string, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp]

In [28]:
predictionstwo = lrModel.transform(datasettest)
predictionstwo

DataFrame[label: double, features: vector, IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: string, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp, rawPrediction: vector, probability: vector, prediction: double]

In [29]:
predictionstwo

DataFrame[label: double, features: vector, IsCanceled: int, LeadTime: int, ArrivalDateYear: int, ArrivalDateMonth: string, ArrivalDateWeekNumber: int, ArrivalDateDayOfMonth: int, StaysInWeekendNights: int, StaysInWeekNights: int, Adults: int, Children: string, Babies: int, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: double, PreviousCancellations: int, PreviousBookingsNotCanceled: int, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: int, DepositType: string, Agent: string, Company: string, DaysInWaitingList: int, CustomerType: string, ADR: double, RequiredCarParkingSpaces: int, TotalOfSpecialRequests: int, ReservationStatus: string, ReservationStatusDate: timestamp, rawPrediction: vector, probability: vector, prediction: double]

In [30]:
type(predictions)

pyspark.sql.dataframe.DataFrame

In [31]:
type(predictionstwo)

pyspark.sql.dataframe.DataFrame

In [32]:
selectedtwo = predictionstwo.select("label", "prediction", "probability")
display(selectedtwo)

DataFrame[label: double, prediction: double, probability: vector]

In [33]:
selected.toPandas().to_csv('predictionsh2_gbt.csv')

## Confusion Matrix

In [34]:
import numpy as np
from numpy.random import seed
seed(1)
from sklearn.metrics import classification_report,confusion_matrix

In [35]:
mydata=pd.read_csv("predictionsh2_gbt.csv")
mydata

Unnamed: 0  label  prediction                               probability
0               0    0.0         0.0  [0.8884304750286717,0.11156952497132833]
1               1    0.0         0.0  [0.8884304750286717,0.11156952497132833]
2               2    0.0         0.0  [0.8766510249655614,0.12334897503443865]
3               3    0.0         0.0  [0.8653444625040015,0.13465553749599846]
4               4    0.0         0.0  [0.8538107273144712,0.14618927268552884]
...           ...    ...         ...                                       ...
11999       11999    1.0         1.0   [0.4984188475175982,0.5015811524824019]
12000       12000    1.0         0.0  [0.8470261544834827,0.15297384551651727]
12001       12001    1.0         0.0  [0.8470261544834827,0.15297384551651727]
12002       12002    1.0         0.0   [0.732965106354776,0.26703489364522404]
12003       12003    1.0         1.0  [0.39341963894607423,0.6065803610539258]

[12004 rows x 4 columns]

In [36]:
label=mydata['label']
prediction=mydata['prediction']

In [37]:
print(confusion_matrix(label,prediction))
print(classification_report(label,prediction))

[[7879  737]
 [1106 2282]]
              precision    recall  f1-score   support

         0.0       0.88      0.91      0.90      8616
         1.0       0.76      0.67      0.71      3388

    accuracy                           0.85     12004
   macro avg       0.82      0.79      0.80     12004
weighted avg       0.84      0.85      0.84     12004

